In [ ]:
import sys
import cv2 as cv
from random import randint
from tqdm import tqdm
import time
import numpy as np
from matplotlib import pyplot as plt
import os
from PIL import Image, ImageFont, ImageDraw
import tensorflow as tf

from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.compat.v1 import graph_util

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

#from object_detection.utils import non_max_suppression
from object_detection.utils import label_map_util
from imutils.object_detection import non_max_suppression


In [ ]:
sys.path

In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

new model

In [ ]:
video = cv.VideoCapture('train/KT_trim.mp4')

In [ ]:
#MODEL_NAME = 'inference_graph'
#MODEL_NAME = ''

In [ ]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
#PATH_TO_CKPT = 'frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [ ]:
label_map = label_map_util.load_labelmap('mscoco_label_map.pbtxt')

In [ ]:
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)

In [ ]:
category_index = label_map_util.create_category_index(categories)

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef() 
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    sess = tf.compat.v1.Session(graph=detection_graph)    

In [ ]:

image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')


In [ ]:
while(True):

    ret, frame = video.read()
    frame_expanded = np.expand_dims(frame, axis=0)

    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: frame_expanded})

    n,p,q = np.shape(frame)
    o,l,m = np.shape(boxes)
    for i in range(l):
        if(scores[0][i] > 0.5 and classes[0][i] == 1):   # 0.5
            ymin, xmin, ymax, xmax = int(boxes[0][i][0] * n), int(boxes[0][i][1] * p), int(boxes[0][i][2] * n), int(boxes[0][i][3] * p)
            cv.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 3)

    cv.namedWindow('frame', cv.WINDOW_NORMAL)
    cv.resizeWindow('frame', 960, 540)
    cv.imshow('frame', frame)

    # Press 'q' to quit
    if cv.waitKey(1) == ord('q'):
        break

video.release()
cv.destroyAllWindows()